## Interval Analysis

In [1]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=-1) # added softmax for probabilities
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()


Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [2]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [3]:
train_model(model, 10)
test_model(model)

Epoch 1/10, Loss: 2.012
Epoch 2/10, Loss: 1.729
Epoch 3/10, Loss: 1.627
Epoch 4/10, Loss: 1.596
Epoch 5/10, Loss: 1.582
Epoch 6/10, Loss: 1.572
Epoch 7/10, Loss: 1.566
Epoch 8/10, Loss: 1.561
Epoch 9/10, Loss: 1.557
Epoch 10/10, Loss: 1.553
Accuracy on images: 92.76


### Write the interval analysis for the simple model

In [4]:
## TODO: Write the interval analysis for the simple model
## you can use https://github.com/Zinoex/bound_propagation
# from bound_propagation import BoundModelFactory, HyperRectangle
def interval_linear(W, b, l_in, u_in):
    W_pos = torch.clamp(W, min=0)
    W_neg = torch.clamp(W, max=0)
    l_out = l_in @ W_pos.T + u_in @ W_neg.T + b
    u_out = u_in @ W_pos.T + l_in @ W_neg.T + b
    return l_out, u_out

def interval_relu(l_in, u_in):
    l_out = F.relu(l_in)
    u_out = F.relu(u_in)
    return l_out, u_out

def interval_forward(model, x, eps):
    
    # input_bounds = HyperRectangle.from_eps(x.view(x.size(0), -1), eps)

    # ibp_bounds = model.ibp(input_bounds)
    # return ibp_bounds
    mean, std = 0.1307, 0.3081
    l = (x - eps - mean) / std
    u = (x + eps - mean) / std
    l = torch.clamp(l, 0, 1)  # clamp within pixel bounds
    u = torch.clamp(u, 0, 1)
    
    # view
    l = l.view(l.size(0), -1)
    u = u.view(u.size(0), -1)

    # fc1
    W1 = model[1].fc.weight
    b1 = model[1].fc.bias
    l, u = interval_linear(W1, b1, l, u)

    # ReLU
    l, u = interval_relu(l, u)

    # fc2
    W2 = model[1].fc2.weight
    b2 = model[1].fc2.bias
    l, u = interval_linear(W2, b2, l, u)

    return l, u

def verify_accuracy(model, test_loader, eps):
    """
    Verified accuracy under l_infinity perturbation of radius eps.
    """
    model.eval()
    verified, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            l_out, u_out = interval_forward(model, images, eps)
            for i in range(images.size(0)):
                true_label = labels[i].item()
                if l_out[i, true_label] > torch.max(u_out[i, torch.arange(10) != true_label]):
                    verified += 1
            total += images.size(0)
    return 100.0 * verified / total

epsilons = np.linspace(0.01, 0.1, 10)
results = []
for eps in epsilons:
    acc = verify_accuracy(model, test_loader, eps)
    results.append((eps, acc))
    print(f"Verified accuracy at eps={eps:.2f}: {acc:.2f}%")


Verified accuracy at eps=0.01: 84.43%
Verified accuracy at eps=0.02: 78.43%
Verified accuracy at eps=0.03: 70.69%
Verified accuracy at eps=0.04: 61.02%
Verified accuracy at eps=0.05: 50.20%
Verified accuracy at eps=0.06: 40.39%
Verified accuracy at eps=0.07: 30.79%
Verified accuracy at eps=0.08: 22.54%
Verified accuracy at eps=0.09: 16.11%
Verified accuracy at eps=0.10: 10.84%
